## TDB10226

In [ ]:
tabella = 'TDB10226'
file = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
result = requests.get(file)
date_column = ['DATA_OSS']
data = pd.read_csv(BytesIO(result.content),compression='zip', header=0, sep=';', quotechar='"', encoding='utf-8',dtype={'ENTE_SEGN':'str', 'FENEC':'str', 'VALORE':'Int32','LOC_SPORT':'Int32'},parse_dates=date_column, dayfirst=False)
data['DATA_OSS'] = pd.to_datetime(data['DATA_OSS'])
# data = data[data['LOC_CTP'].isin(nuts1)]
# data = data[data['DATA_OSS'] == data['DATA_OSS'].max()]
data['DATA_OSS'] = data['DATA_OSS'].dt.date
data = pd.merge(data, stamen, how = 'left', left_on='ENTE_SEGN', right_on='Elemento').drop(columns=['ENTE_SEGN','STATUS','FENEC','index','Dominio','Elemento']).rename(columns={'Descrizione': 'segnalante'})
data = pd.merge(data, stamen, how = 'left', left_on='LOC_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'area'})
data = pd.merge(data, stamen, how = 'left', left_on='SET_CTP', right_on='Elemento').drop(columns=['index','Dominio', 'Elemento']).rename(columns={'Descrizione': 'target'})
# data = data[['DATA_OSS', 'LOC_CTP', 'area', 'SET_CTP','target','VALORE']]
data.to_sql(tabella, conn, if_exists='replace', index=False)

data.to_parquet(f"D:/{tabella}.parquet")

data.to_sql(tabella, conn, if_exists='replace', index=False)

conn.execute(f"CREATE TABLE IF NOT EXISTS {tabella} AS SELECT * FROM data LIMIT 0")  # Crea una tabella vuota con la struttura del DataFrame
conn.execute(f"INSERT INTO {tabella} SELECT * FROM data")

sheet_to_update = tabella
book = load_workbook(file_path)
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    data.to_excel(writer, sheet_name=sheet_to_update, index=False)

In [2]:
from sqlalchemy import create_engine
import pandas as pd, sqlite3
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.0f}".format

from io import BytesIO
import requests
import pandas as pd
import os
import sqlalchemy
sqlite = create_engine('sqlite:///D:/files/Bankit.sqlite')
import sqlite3
import sys

## TDB10226 - impieghi vivi per localizzazione della clientela
Prestiti (esclusi PCT e sofferenze) - per provincia e settore della clientela

In [3]:
tabella = 'TDB10226'
data = pd.read_sql_table(tabella, 'sqlite:///D:/files//Bankit.sqlite')

In [36]:
periodo = data[data['DATA_OSS'].isin([pd.Timestamp("2016-12-31"), pd.Timestamp("2023-12-31"), pd.Timestamp("2024-12-31")])]
area_periodo = data[
    (data['DATA_OSS'].isin(periodo['DATA_OSS'])) & 
    (data['SET_CTP'].isin(["SBI25","S11BI13","SBI33"]))]
df = pd.pivot_table(area_periodo, index = 'area', columns = ['target','DATA_OSS'], values = 'VALORE')

abr = [53,27,84,111,0]
mol = [23,46,72]
cam = [74,7,12,24,101,22]
pug = [8,9,18,38,57,110,92]
bas = [90,68,10]
cal = [96,26,29,31,129,20]
sic = [79,1,21,25,34,69,93,107,115,105]
sar = [19, 76,77,103,109, 102]
reg = [0,72,22,92,10,20,105,102]
ita = [47,48,49,50,51,52]

ordine_custom = ita
ordine_custom_colonne = [6,7,8,0,1,2,3,4,5]  # Riorganizza le colonne
df.reset_index(inplace=False)
df = df.iloc[:, ordine_custom_colonne]
df.iloc[ordine_custom]

target                  Società non finanziarie e famiglie produttrici  \
DATA_OSS                                                    2016-12-31   
area                                                                     
Isernia                                                        316,751   
Italia                                                     701,184,756   
Italia centrale                                            154,275,949   
Italia insulare                                             28,096,194   
Italia meridionale                                          68,383,229   
Italia nord-occidentale                                    262,460,581   

target                                           \
DATA_OSS                 2023-12-31  2024-12-31   
area                                              
Isernia                     452,644     482,101   
Italia                  664,918,996 641,480,133   
Italia centrale         132,928,808 129,564,774   
Italia insulare          25,883,436  24,975,191   
Italia meridionale       69,828,798  67,199,984   
Italia nord-occidentale 255,747,758 250,577,080   

target                  Società di persone, semplici, di fatto e ditte individuali con meno di 20 addetti  \
DATA_OSS                                                                                       2016-12-31   
area                                                                                                        
Isernia                                                             98,225                                  
Italia                                                         128,998,999                                  
Italia centrale                                                 25,012,686                                  
Italia insulare                                                  7,656,120                                  
Italia meridionale                                              16,130,975                                  
Italia nord-occidentale                                         39,709,515                                  

target                                          \
DATA_OSS                 2023-12-31 2024-12-31   
area                                             
Isernia                      84,971     72,455   
Italia                  108,022,985 99,524,564   
Italia centrale          20,709,320 19,029,057   
Italia insulare           7,540,042  7,121,696   
Italia meridionale       15,574,755 14,308,084   
Italia nord-occidentale  32,351,499 29,781,513   

target                  Società non finanziarie  con almeno  20 addetti  \
DATA_OSS                                                     2016-12-31   
area                                                                      
Isernia                                                         218,526   
Italia                                                      572,185,758   
Italia centrale                                             129,263,264   
Italia insulare                                              20,440,075   
Italia meridionale                                           52,252,254   
Italia nord-occidentale                                     222,751,066   

target                                           
DATA_OSS                 2023-12-31  2024-12-31  
area                                             
Isernia                     367,673     409,647  
Italia                  556,896,011 541,955,569  
Italia centrale         112,219,488 110,535,717  
Italia insulare          18,343,394  17,853,494  
Italia meridionale       54,254,043  52,891,900  
Italia nord-occidentale 223,396,258 220,795,567

In [ ]:
area_periodo.query('SET_CTP == "SBI25"').groupby(['area', 'DATA_OSS'])['VALORE'].sum().reset_index().sort_values(by=['area', 'DATA_OSS'], ascending=[True, False])

In [ ]:
pd.pivot_table(area_periodo, index = ['area','DATA_OSS'], columns = 'SET_CTP', values = 'VALORE')

## TDB10255

In [3]:
tabella = 'TDB10255'
data = pd.read_sql_table(tabella, 'sqlite:///D:/files//Bankit.sqlite')

In [5]:
data.sample()

,ATECO_CTP,DATA_OSS,ENTE_SEGN,FENEC,LOC_CTP,SET_CTP,VALORE,STATUS
7305,G,2015-03-31,1070001,1077778,ITI4,SBI25,13368969,NaN
